In [9]:
import time




import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/janani/Downloads/coding/pyspark-project/elasticsearch-hadoop-7.12.0.jar pyspark-shell'

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.types import (StructField, StringType, StructType, IntegerType)

start = time.time()

conf = pyspark.SparkConf().setAppName('SparkLog').setMaster('local')
sc = pyspark.SparkContext.getOrCreate(conf=conf)
schema1 =  StructType([ StructField("Date",StringType(), True), \
         StructField("Time",StringType(), True), \
         StructField("Level",StringType(), True),\
         StructField("Msg",StringType(), True)
         ])


logpath = 'Android.log' #Need to change this logpath for different log types
logTxt = sc.textFile(logpath)
logs = logTxt.map(lambda k: [k.split()[0], k.split()[1], k.split()[2], ''.join(k.split()[3:])])
logs.take(5)



[['12-17',
  '19:31:36.263',
  '1795',
  '1825IPowerManager_screenOn:DisplayPowerStatesetColorFadeLevel:level=1.0'],
 ['12-17',
  '19:31:36.263',
  '5224',
  '5283ISendBroadcastPermission:action:android.com.huawei.bone.NOTIFY_SPORT_DATA,mPermissionType:0'],
 ['12-17',
  '19:31:36.264',
  '1795',
  '1825DDisplayPowerController:Animatingbrightness:target=21,rate=40'],
 ['12-17',
  '19:31:36.264',
  '1795',
  '1825IPowerManager_screenOn:DisplayPowerControllerupdatePowerStatemPendingRequestLocked=policy=BRIGHT,useProximitySensor=true,useProximitySensorbyPhone=true,screenBrightness=33,screenAutoBrightnessAdjustment=0.0,brightnessSetByUser=true,useAutoBrightness=true,blockScreenOn=false,lowPowerMode=false,boostScreenBrightness=false,dozeScreenBrightness=-1,dozeScreenState=UNKNOWN,useTwilight=false,useSmartBacklight=true,brightnessWaitMode=false,brightnessWaitRet=true,screenAutoBrightness=-1,userId=0'],
 ['12-17',
  '19:31:36.264',
  '1795',
  '2750IPowerManager_screenOn:DisplayPowerStateUpda

In [10]:
import json
import hashlib
import re

def addId(data):
    j=json.dumps(data).encode('ascii', 'ignore')
    data['doc_id'] = hashlib.sha224(j).hexdigest()
    return (data['doc_id'], json.dumps(data))

def parse1(item):
    d = {}
    d['date']=item[0]
    d['time']=item[1]
    d['level']=item[2]
    d['msg']=item[3]
    return d

In [ ]:
rdd = logs.map(parse1)
final_rdd = rdd.map(addId)

end = time.time()
print(end - start)

final_rdd.saveAsNewAPIHadoopFile(path='-',
outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
keyClass="org.apache.hadoop.io.NullWritable",
valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
conf={ "es.resource" : "spark_spark/doc", "es.mapping.id":"id",
"es.input.json": "true", "es.net.http.auth.user":"elastic",
"es.write.operation":"index", "es.nodes.wan.only":"false",
"es.net.http.auth.pass":"nYCy8JB8xQ3ImulyQGNp", "es.nodes":"http://ms1133.utah.cloudlab.us",
"es.port":"3718" })